Analisis Exploratorio de Datos: EDA

In [3]:
import numpy as np
import yfinance as yf
import pandas as pd


In [10]:
sp500_data = yf.download('^GSPC', start='2000-01-01')


[*********************100%***********************]  1 of 1 completed


In [11]:
sp500_data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-03,1469.250000,1478.000000,1438.359985,1455.219971,1455.219971,931800000
2000-01-04,1455.219971,1455.219971,1397.430054,1399.420044,1399.420044,1009000000
2000-01-05,1399.420044,1413.270020,1377.680054,1402.109985,1402.109985,1085500000
2000-01-06,1402.109985,1411.900024,1392.099976,1403.449951,1403.449951,1092300000
2000-01-07,1403.449951,1441.469971,1400.729980,1441.469971,1441.469971,1225200000


In [9]:
sp500_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5822 entries, 2000-01-03 to 2023-02-22
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       5822 non-null   float64
 1   High       5822 non-null   float64
 2   Low        5822 non-null   float64
 3   Close      5822 non-null   float64
 4   Adj Close  5822 non-null   float64
 5   Volume     5822 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 318.4 KB
